In [1]:
from IPython.display import display, HTML

# import matplotlib and numpy
import matplotlib.pyplot as plt 
import matplotlib.image as mpimage
import matplotlib.patches as patches

from matplotlib import cm
import colorcet as cc
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1 import make_axes_locatable

import numpy as np 
import pandas as pd
import cv2

# import operating system and glob libraries
import os

import time
from datetime import datetime

import scipy
from scipy import ndimage

from pathlib import Path

# this following line tells Jupyter to display images here in the browser, 
# rather than in separate window.
%matplotlib inline 

dicomImage_issues = []
observationImage_issues = []

In [2]:
%%capture
%run numpngw.ipynb
%load_ext line_profiler

In [3]:
cmap_grey = cc.cm.linear_grey_0_100_c0

In [4]:
CBIS_DDSM_dir = "/Users/kasparlund/AICodeData/x-ray/CBIS-DDSM-mammography/"
converted_dir          = CBIS_DDSM_dir+"converted_images/"
converted_png          = converted_dir+"png/"

case_data_filepath = converted_dir+"converted_data_with_dimensions.csv"
cases = pd.read_csv(case_data_filepath, sep=";")


gp = cases.groupby( ["purpose", "simple_pathology"])["case_id"].count()
print( gp )


purpose  simple_pathology          
test     BENIGN                         381
         BENIGN;MALIGNANT                 4
         MALIGNANT                      260
train    BENIGN                        1346
         BENIGN;MALIGNANT                 5
         BENIGN;MALIGNANT;MALIGNANT       2
         MALIGNANT                     1097
         MALIGNANT;BENIGN                 6
Name: case_id, dtype: int64


In [5]:
equalized_dir       = CBIS_DDSM_dir+"equalized2dogscatshist/full_images/"
tile_dir            = CBIS_DDSM_dir+"equalized2dg_tiles/"

In [6]:
def simplePlot(img, title, cmap, figsize=(8,8)):
    fig = plt.figure(figsize=figsize )
    plt.title(title)
    plt.imshow(img, cmap)    
    plt.show()

In [7]:
def doublePlot(mainTitle, title1, img1, title2, img2, colormap, width = 16 ):

    height = round( (width*img1.shape[0]) /img1.shape[1] )

    fig = plt.figure(figsize = (width,height)) 
    plt.tight_layout()
    #fig.suptitle(mainTitle)

    ax1 = fig.add_subplot(221)
    ax1.set_title(title1)
    im1 = ax1.imshow(img1, cmap=colormap)
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes("bottom", "5%", pad="3%")
    colorbar = fig.colorbar(im1, cax=cax, orientation="horizontal")

    ax2 = fig.add_subplot(222)
    ax2.set_title(title2)

    im2 = ax2.imshow(img2, cmap=colormap)
    divider2 = make_axes_locatable(ax2)
    cax2 = divider2.append_axes("bottom", "5%", pad="3%")
    colorbar2 = fig.colorbar(im2, cax=cax2, orientation="horizontal")


    plt.show()    
    return

In [8]:
def plotHistogramsBeforeAndAfter(title, before, after, limits=[0.01, 0.99], x=None):
    fig=plt.figure(figsize = (12,12), dpi=100) 
    plt.suptitle(title, fontsize=15)
    plt.tight_layout()

    nb_plot_bins = 512
    ax1 = fig.add_subplot(221)
    ax1.set_title("before")
    plt.hist(before.flatten(), nb_plot_bins, limits )

    ax2 = fig.add_subplot(222)
    ax2.set_title("After: " )
    plt.hist(after.flatten(), nb_plot_bins, limits )
    plt.show()

In [9]:
def centerOfObs(mask):
    img,contours,hierarchy = cv2.findContours( mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE )
    centers = []
    bbs = []
    area = []
    # loop over the contours
    for c in contours:
        #ignore isonlated 1 pixels at the border og the observation
        if c.shape[0] < 3 : continue 

        c  = np.squeeze(c)
        bb = cv2.boundingRect(c)
        #arrange the bb totopleft , bottom right matrix coordinates
        bb = np.asarray([bb[i] for i in [1,0,3,2]])
        
        # compute the center of the contour
        moments = cv2.moments(c)
        m00 = moments["m00"]
        m10 = moments["m10"]
        m01 = moments["m01"]
        #print( f"{m00}, {m10}, {m01}")
        if m00 > 0:
            cX = int(m10 / m00)
            cY = int(m01 / m00)
            centers.append([cY,cX])
            bbs.append( bb )
            area.append(int(m00))
            #cv2.circle(img, (cX, cY), 7, (0, 0, 0), -1)
            #print( f"cX:{cX}, cY:{cY}")
        #cv2.rectangle(img, (bb[0],bb[1]), (bb[0]+bb[2], bb[1]+bb[3]), (255, 255, 255), cv2.LINE_8);
        #print(contours)
    #simplePlot(img, "center of mask", cmap_grey)
    #doublePlot("centerOfObs","mask",mask,  "image from countour", img, cmap_grey, width = 6)
    return np.asarray(centers), np.asarray(bbs), np.asarray(area)

In [10]:
#def pixel_ratio(im,value): return round(np.sum(im.flatten()==value)/(im.shape[0]*im.shape[1]), 3)
def pixel_count(im,value): return np.sum(im.flatten()==value)

In [11]:
def offset2Tiles(of, im_dir, ma_dir, testOrTrain, case_id, im, mask, sz, obsArea, ignoreObs=False, min_Air=0.5):
    case_ids,tile_ids,fnImages, fnMasks, pAir,pHealthy,pBenign,\
        pMalignant,rCenter,cCenter,tile_size, obsAreas = [],[],[],[],[],[],[],[],[],[],[],[]

    noTilePixels = sz[0]*sz[1]
    for o in of:
        im_t  = im[o[0]:o[0]+sz[0], o[1]:o[1]+sz[1]]
        p_Air = pixel_count(im_t,0)
        #ignore tiles that are mostly air
        if p_Air/noTilePixels > min_Air : continue
            
        ma    = mask[o[0]:o[0]+sz[0], o[1]:o[1]+sz[1]]
        p_Benign, p_Malignant = pixel_count(ma,128), pixel_count(ma,255)
        p_Healthy = noTilePixels - p_Air - p_Benign - p_Malignant
        
        #if ignoreObs then skip tiles with benign og malignant tissue 
        if ignoreObs and (p_Benign+p_Malignant)/noTilePixels > 0.0: continue
            
        tile_id    = f"{case_id}_tile_{o[0]}_{o[1]}" 
        image_file = testOrTrain + "/" + tile_id +".png" #f"{im_dir}/{tile_id}.png"
        mask_file  = testOrTrain + "/" + tile_id +".png" #f"{ma_dir}/{tile_id}.png"

        case_ids.append(case_id )
        tile_ids.append(   tile_id)
        fnImages.append(   image_file )
        fnMasks.append(    mask_file )
        pAir.append(       p_Air )
        pHealthy.append(   p_Healthy )
        pBenign.append(    p_Benign )
        pMalignant.append( p_Malignant )
        rCenter.append( round( (o[0]+0.5*sz[0])/im.shape[0] ,2) )
        cCenter.append( round( (o[1]+0.5*sz[1])/im.shape[1] ,2) )
        tile_size.append(  noTilePixels )
        obsAreas.append(    obsArea )
        cv2.imwrite( im_dir+"/"+ image_file, (65535.0*im_t + 0.5).astype(np.uint16) )    
        cv2.imwrite( ma_dir+"/"+ mask_file, ma )

        #if o[0]==672 and o[1]==0 :
        #    print(f"offset:{o[0]},{o[1]}: healthy: {pHealthy} air :{pAir}, benign: {pBening} malignant: {pMalignant}")
        #    #simplePlot(t,f"tile: {o}",cmap_grey, figsize=(4,4))
        #    doublePlot("tiles","im",t, "mask", m, cmap_grey, width = 8)

        #print(f"r:{o[0]}:{o[0]+sz[0]} c:{o[1]}:{o[1]+sz[1]}")
    result = pd.DataFrame( 
        np.column_stack([
                    case_ids, tile_ids, pAir,  pHealthy, \
                    pBenign, pMalignant, rCenter, cCenter, tile_size, obsAreas,  fnImages, fnMasks ]),
        columns = ["case_id","tile_id","pAir","pHealthy", \
                   "pBenign","pMalignant","rCenter","cCenter","tile_size","obsAreas","fnImage","fnMask"])
        
    return result

In [12]:
def imageCoord(xy_arrayIndex) : return (xy_arrayIndex[1], xy_arrayIndex[0] )
#sz :size of the tiles
def generate_tile2file( im_dir, ma_dir, testOrTrain, case_id, im, mask, tilesPRObservation, sz=[224,224] ):
    sz = np.asarray(sz)

    #generate tiles on a regular grid
    nb = np.ceil(im.shape / np.asarray(sz)).astype(int)
    last_of = im.shape - sz
    #calculate alle offset
    of = []
    for i in range(0,nb[0]):
        r = i*sz[0] if i+1<nb[0] else last_of[0]
        for j in range(0,nb[1]):
            c = j*sz[1] if j+1<nb[1] else last_of[1]
            of.append((r,c)) 
    #print( f"nb tiles:{len(of)} of: {of}")
    #get alle tile that do not overlap with observations (ignoreObs=True)
    allResult = offset2Tiles(of, im_dir, ma_dir, testOrTrain, case_id, im, mask, sz, obsArea=0, ignoreObs=True)

    #generate tiles that overlap the observations
    centers,bbs, area = centerOfObs(mask)
    for b,a in zip(bbs,area): 
        inset  = np.minimum(sz//2, b[2:4]//2)
        tl_min = b[0:2] + inset - sz
        tl_min = np.maximum(tl_min, np.zeros_like(tl_min))
        tl_max = b[0:2] + b[2:4] - inset
        tl_max = np.minimum(tl_max+sz, im.shape) -sz
        #print("b: ", b, " tl_min: ", tl_min, " tl_max: ",tl_max, " inset: ", inset, "area:", round(a/(sz[0]*sz[1]),2) )

        of = np.random.uniform( low=tl_min, high=tl_max, size=[tilesPRObservation,2] ).astype(int)
        result = offset2Tiles(of, im_dir, ma_dir, testOrTrain, case_id, im, mask, sz, obsArea=a, )
        if allResult is None:    allResult = result 
        elif result is not None: allResult = allResult.append( result )
            
        #print("of:\n",of)
        #cv2.rectangle(mask, pt1=imageCoord(tl_min), pt2=imageCoord(tl_min+sz), color=(255, 255, 255), thickness=4, lineType=cv2.LINE_8 )
        #cv2.rectangle(mask, pt1=imageCoord(tl_max), pt2=imageCoord(tl_max+sz), color=(255, 255, 255), thickness=4, lineType=cv2.LINE_8 )
        """
        for o in of:
            tl = o
            lr = tl + sz
            print("tl: ", tl, " lr: ",lr)
            cv2.rectangle(mask, pt1=imageCoord(tl), pt2=imageCoord(lr), color=(255, 255, 255), 
                          thickness=4, lineType=cv2.LINE_8 )
        #cv2.rectangle(im, (tl[0],tl[1]), (lr[0], lr[1]), (255, 255, 255), cv2.LINE_8);
#   doublePlot("tiles","im",im, "mask", mask, cmap_grey, width = 16)
    simplePlot(mask, "tiles",cmap_grey, figsize =(16,16))
        """        
    return allResult


In [13]:
def generateTiles( tileCases, scaling ):
    os.makedirs(tile_dir + "/tiles/train" , exist_ok=True)
    os.makedirs(tile_dir + "/tiles/test" ,  exist_ok=True)
    os.makedirs(tile_dir + "/masks/train" , exist_ok=True)
    os.makedirs(tile_dir + "/masks/test" ,  exist_ok=True)
    
    dfTiles = None
    nb_cases = tileCases.shape[0]
    for i in range(0,nb_cases):
        r = tileCases.iloc[i]
    
        testOrTrain = "test" if r.purpose=="test" else "train"
        s_x_ray     = equalized_dir + testOrTrain + "/" + r.case_id + ".png" 
        s_x_mask    = converted_png + testOrTrain + "/main_obs_masks/"   + r.case_id + "_alle_obs.png"        
        t_tile_dir  = tile_dir + "/tiles/" 
        m_tile_dir  = tile_dir + "/masks/" 
        
        im    = cv2.imread(s_x_ray,  cv2.IMREAD_ANYDEPTH)/65536.0
        mask  = cv2.imread(s_x_mask, cv2.IMREAD_ANYDEPTH).astype(np.uint8)
    
        if im.shape[1] >= 448:
            dsize =(round(im.shape[1]*scaling),round(im.shape[0]*scaling))
            im    = cv2.resize(im,   dsize=dsize, interpolation=cv2.INTER_AREA)
            mask  = cv2.resize(mask, dsize=dsize, interpolation=cv2.INTER_NEAREST)
        elif im.shape[1] < 224: continue
       
        #print("nb values==0: ", np.sum(im==0.0), "nb values==0.01: ", np.sum(im==0.01))
        #print("nb values==0: ", np.sum(im2==0.0), "nb values==0.01: ", np.sum(im2==0.01))
        #print("u_before: ", len(np.unique(im)), " u_after: ", len(np.unique(im2)) )
        #doublePlot(r.case_id,"x-ray before",im,  "x_ray with global equalization", mask, cmap_grey, width = 12)
        #doublePlot(r.case_id,"x-ray before",im2, "x_ray with global equalization", mask, cmap_grey, width = 12)
    
        prObservation=15
        result = generate_tile2file( t_tile_dir, m_tile_dir, testOrTrain, r.case_id, im, mask, tilesPRObservation=prObservation )
        if dfTiles is None:    dfTiles = result 
        elif result is not None: dfTiles = dfTiles.append( result )
        
        if i-(int(i/50)*50) == 0 :
            percentage = int(np.around( i*100 / nb_cases))
            print( f"current case: {r.case_id}  Processed: {percentage}%  Generated tiles: ", dfTiles.shape[0] if dfTiles is not None else 0 )
    return dfTiles

In [14]:
"""
testcases =  [
#"Mass-Training_P_00797_LEFT_CC",
#"Mass-Training_P_00419_LEFT_MLO",
#"Mass-Test_P_00969_LEFT_CC",
#"Calc-Training_P_00937_RIGHT_MLO",
#"Mass-Training_P_00997_LEFT_CC" #the algo get width wrong because it is less<224
"Calc-Test_P_00038_LEFT_CC"
]
cases = cases[ np.isin( cases.case_id, testcases ) ]
"""
#tile_cases = cases[ np.isin( cases.purpose, ["training"x] ) ]

start = time.clock()
dfTiles = generateTiles(cases,scaling = 0.5)
print("runTime ", time.clock() - start )

if dfTiles is not None: print("total number og tiles generated: ", dfTiles.shape)


current case: Calc-Test_P_00038_LEFT_CC  Processed: 0%  Generated tiles:  33
current case: Calc-Test_P_00353_LEFT_CC  Processed: 2%  Generated tiles:  2778
current case: Calc-Test_P_00753_LEFT_MLO  Processed: 3%  Generated tiles:  5165
current case: Calc-Test_P_01157_RIGHT_CC  Processed: 5%  Generated tiles:  7512
current case: Calc-Test_P_01483_LEFT_CC  Processed: 6%  Generated tiles:  9750
current case: Calc-Test_P_01868_LEFT_CC  Processed: 8%  Generated tiles:  12188
current case: Calc-Training_P_00014_LEFT_MLO  Processed: 10%  Generated tiles:  14973
current case: Calc-Training_P_00098_RIGHT_MLO  Processed: 11%  Generated tiles:  17399
current case: Calc-Training_P_00232_RIGHT_CC  Processed: 13%  Generated tiles:  20187
current case: Calc-Training_P_00307_LEFT_CC  Processed: 15%  Generated tiles:  22658
current case: Calc-Training_P_00398_LEFT_MLO  Processed: 16%  Generated tiles:  25070
current case: Calc-Training_P_00479_LEFT_MLO  Processed: 18%  Generated tiles:  27916
current c

In [15]:

if dfTiles is not None:
    tile_cases = pd.merge(cases, dfTiles, how="right", left_on='case_id', right_on='case_id')
    tile_cases = tile_cases.drop(["obs_id"],axis=1)
    tile_cases.to_csv(tile_dir + "/tiles.csv", sep=";", index=False)
    print("tile_cases.shape: ",tile_cases.shape, " dfTiles.shape: ", dfTiles.shape)


tile_cases.shape:  (161117, 28)  dfTiles.shape:  (161117, 12)
